# 📊 Financial Research Analyst Agent - Exploration Notebook

This notebook demonstrates the capabilities of the Financial Research Analyst Agent system.

## Contents
1. Setup and Configuration
2. Data Collection Demo
3. Technical Analysis Demo
4. Fundamental Analysis Demo
5. Sentiment Analysis Demo
6. Full Agent Analysis
7. Portfolio Analysis

## 1. Setup and Configuration

In [ ]:
# Install required packages (if needed)
# !pip install -r ../requirements.txt

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

# Import project modules
from src.tools.market_data import get_stock_price, get_historical_data, get_company_info
from src.tools.technical_indicators import (
    calculate_rsi, calculate_macd, calculate_moving_averages, calculate_bollinger_bands
)
from src.tools.financial_metrics import (
    calculate_valuation_ratios, calculate_profitability_ratios, analyze_financial_health
)

print("✅ Modules imported successfully!")

## 2. Data Collection Demo

In [ ]:
# Fetch current stock price
symbol = "AAPL"
price_data = get_stock_price(symbol)

print(f"📈 {symbol} Current Data:")
print(f"   Price: ${price_data.get('current_price', 0):.2f}")
print(f"   Change: {price_data.get('change_percent', 0):.2f}%")
print(f"   Volume: {price_data.get('volume', 0):,}")
print(f"   Market Cap: ${price_data.get('market_cap', 0):,.0f}")

In [ ]:
# Fetch historical data
hist_data = get_historical_data(symbol, period="6mo")

# Create DataFrame
df = pd.DataFrame({
    'Date': pd.to_datetime(hist_data['dates']),
    'Open': hist_data['opens'],
    'High': hist_data['highs'],
    'Low': hist_data['lows'],
    'Close': hist_data['closes'],
    'Volume': hist_data['volumes']
})
df.set_index('Date', inplace=True)

print(f"📊 Historical Data: {len(df)} data points")
df.tail()

In [ ]:
# Plot price chart
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=df['Close'],
    name='Price'
))
fig.update_layout(
    title=f'{symbol} Price Chart (6 Months)',
    yaxis_title='Price ($)',
    template='plotly_dark',
    height=500
)
fig.show()

## 3. Technical Analysis Demo

In [ ]:
# Calculate RSI
closes = hist_data['closes']
rsi_result = calculate_rsi(closes)

print("📊 RSI Analysis:")
print(f"   RSI Value: {rsi_result['value']:.2f}")
print(f"   Signal: {rsi_result['signal']}")
print(f"   Interpretation: {rsi_result['interpretation']}")

In [ ]:
# Calculate MACD
macd_result = calculate_macd(closes)

print("📊 MACD Analysis:")
print(f"   MACD Line: {macd_result['macd_line']:.4f}")
print(f"   Signal Line: {macd_result['signal_line']:.4f}")
print(f"   Histogram: {macd_result['histogram']:.4f}")
print(f"   Trend: {macd_result['trend'].upper()}")
print(f"   Crossover: {macd_result['crossover']}")

In [ ]:
# Calculate Moving Averages
ma_result = calculate_moving_averages(closes)

print("📊 Moving Averages:")
print(f"   Current Price: ${ma_result['current_price']:.2f}")
print(f"   SMA 20: ${ma_result.get('sma_20', 0):.2f}")
print(f"   SMA 50: ${ma_result.get('sma_50', 0):.2f}")
print(f"   SMA 200: ${ma_result.get('sma_200', 0):.2f}")
if 'trend' in ma_result:
    print(f"   Trend: {ma_result['trend'].upper()}")

In [ ]:
# Calculate Bollinger Bands
bb_result = calculate_bollinger_bands(closes)

print("📊 Bollinger Bands:")
print(f"   Upper Band: ${bb_result['upper_band']:.2f}")
print(f"   Middle Band: ${bb_result['middle_band']:.2f}")
print(f"   Lower Band: ${bb_result['lower_band']:.2f}")
print(f"   Bandwidth: {bb_result['bandwidth']:.2f}%")
print(f"   Position: {bb_result['position']}")

In [ ]:
# Plot technical indicators
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Price with Moving Averages
axes[0].plot(df.index, df['Close'], label='Close', color='white', linewidth=1.5)
axes[0].set_title(f'{symbol} Technical Analysis', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Price ($)')
axes[0].legend()
axes[0].grid(alpha=0.3)

# RSI
# Calculate RSI series for plotting
def calc_rsi_series(prices, period=14):
    deltas = np.diff(prices)
    gains = np.where(deltas > 0, deltas, 0)
    losses = np.where(deltas < 0, -deltas, 0)
    
    rsi_values = [np.nan] * period
    avg_gain = np.mean(gains[:period])
    avg_loss = np.mean(losses[:period])
    
    for i in range(period, len(deltas)):
        avg_gain = (avg_gain * (period - 1) + gains[i]) / period
        avg_loss = (avg_loss * (period - 1) + losses[i]) / period
        rs = avg_gain / avg_loss if avg_loss != 0 else 100
        rsi_values.append(100 - (100 / (1 + rs)))
    
    return [np.nan] + rsi_values

rsi_series = calc_rsi_series(closes)
axes[1].plot(df.index[-len(rsi_series):], rsi_series, label='RSI', color='#8b5cf6', linewidth=1.5)
axes[1].axhline(y=70, color='red', linestyle='--', alpha=0.5, label='Overbought')
axes[1].axhline(y=30, color='green', linestyle='--', alpha=0.5, label='Oversold')
axes[1].set_ylabel('RSI')
axes[1].set_ylim(0, 100)
axes[1].legend()
axes[1].grid(alpha=0.3)

# Volume
colors = ['green' if df['Close'].iloc[i] >= df['Open'].iloc[i] else 'red' for i in range(len(df))]
axes[2].bar(df.index, df['Volume'], color=colors, alpha=0.7)
axes[2].set_ylabel('Volume')
axes[2].set_xlabel('Date')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.style.use('dark_background')
plt.show()

## 4. Fundamental Analysis Demo

In [ ]:
# Get company info
company_info = get_company_info(symbol)

print(f"🏢 Company Information: {symbol}")
print(f"   Name: {company_info.get('name', 'N/A')}")
print(f"   Sector: {company_info.get('sector', 'N/A')}")
print(f"   Industry: {company_info.get('industry', 'N/A')}")
print(f"   Employees: {company_info.get('employees', 0):,}")
print(f"   Market Cap: ${company_info.get('market_cap', 0):,.0f}")

In [ ]:
# Calculate valuation ratios
valuation_data = {
    'price': price_data.get('current_price', 0),
    'eps': price_data.get('eps', 0),
    'book_value_per_share': 20,  # Example
    'revenue_per_share': 25,  # Example
    'enterprise_value': company_info.get('enterprise_value', 0),
    'ebitda': 100e9,  # Example
}

valuation = calculate_valuation_ratios(valuation_data)

print("📊 Valuation Ratios:")
for key, value in valuation.items():
    print(f"   {key}: {value}")

## 5. Comprehensive Analysis Summary

In [ ]:
# Generate comprehensive analysis summary
print("=" * 70)
print(f"           COMPREHENSIVE ANALYSIS REPORT: {symbol}")
print("=" * 70)
print()
print(f"Company: {company_info.get('name', symbol)}")
print(f"Sector: {company_info.get('sector', 'N/A')}")
print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")
print()
print("-" * 70)
print("PRICE DATA")
print("-" * 70)
print(f"Current Price: ${price_data.get('current_price', 0):.2f}")
print(f"52-Week High: ${price_data.get('52_week_high', 0):.2f}")
print(f"52-Week Low: ${price_data.get('52_week_low', 0):.2f}")
print()
print("-" * 70)
print("TECHNICAL INDICATORS")
print("-" * 70)
print(f"RSI (14): {rsi_result['value']:.2f} - {rsi_result['signal']}")
print(f"MACD Trend: {macd_result['trend'].upper()}")
print(f"Bollinger Position: {bb_result['position']}")
print()
print("-" * 70)
print("RECOMMENDATION")
print("-" * 70)

# Generate simple recommendation based on indicators
score = 0
if rsi_result['value'] < 30:
    score += 1
elif rsi_result['value'] > 70:
    score -= 1
    
if macd_result['trend'] == 'bullish':
    score += 1
elif macd_result['trend'] == 'bearish':
    score -= 1

if score >= 1:
    recommendation = "BUY"
    confidence = 0.6 + (score * 0.1)
elif score <= -1:
    recommendation = "SELL"
    confidence = 0.6 + (abs(score) * 0.1)
else:
    recommendation = "HOLD"
    confidence = 0.5

print(f"Action: {recommendation}")
print(f"Confidence: {confidence*100:.0f}%")
print()
print("=" * 70)

## 6. Portfolio Analysis

In [ ]:
# Analyze multiple stocks
portfolio = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'META']

portfolio_data = []
for sym in portfolio:
    try:
        data = get_stock_price(sym)
        hist = get_historical_data(sym, period="1y")
        rsi = calculate_rsi(hist['closes'])
        
        portfolio_data.append({
            'Symbol': sym,
            'Price': data.get('current_price', 0),
            'Change %': data.get('change_percent', 0),
            'Market Cap (B)': data.get('market_cap', 0) / 1e9,
            'P/E': data.get('pe_ratio', 0),
            'RSI': rsi.get('value', 0),
            'RSI Signal': rsi.get('signal', 'N/A'),
        })
    except Exception as e:
        print(f"Error fetching {sym}: {e}")

portfolio_df = pd.DataFrame(portfolio_data)
portfolio_df

In [ ]:
# Visualize portfolio
fig = px.bar(
    portfolio_df, 
    x='Symbol', 
    y='RSI', 
    color='RSI Signal',
    title='Portfolio RSI Analysis',
    color_discrete_map={
        'OVERSOLD': '#10b981',
        'NEUTRAL': '#f59e0b',
        'OVERBOUGHT': '#ef4444'
    },
    template='plotly_dark'
)
fig.add_hline(y=70, line_dash="dash", line_color="red", annotation_text="Overbought")
fig.add_hline(y=30, line_dash="dash", line_color="green", annotation_text="Oversold")
fig.show()

## 🎉 Conclusion

This notebook demonstrated the core capabilities of the Financial Research Analyst Agent:

1. **Data Collection**: Fetching real-time and historical market data
2. **Technical Analysis**: Calculating RSI, MACD, Moving Averages, Bollinger Bands
3. **Fundamental Analysis**: Evaluating valuation ratios and company metrics
4. **Portfolio Analysis**: Analyzing multiple stocks simultaneously

For the full agent experience with LLM-powered insights, run the API server:
```bash
python -m src.main api
```